In [1]:
import torch

#imports necessary to define a neural network 
import torch.nn as nn
import torch.nn.functional as F

#ensure you are using GPU.
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  

device = torch.device(dev)
print(device)

dtype = torch.cuda.FloatTensor

cuda:0


In [2]:
a0=9.0
a1=8.97
a2=0.876
a3=2.9876

def output(x0,x1,x2,x3):
    
    out=a0*x0**(0.87)
    out=out+a1*x1**(0.020)
    out=out+a2*x2**(0.12)
    out=out+a3*x3**(0.987)
    
    return out

In [3]:
xtrain=1000.0*torch.rand(10000,4).type(dtype)
xtest=1000.0*torch.rand(1000,4).type(dtype)

In [4]:
ytest=torch.empty(1000,1).type(dtype)

for i in range(1000):
    
    ytest[i]=output(xtest[i][0],xtest[i][1],xtest[i][2],xtest[i][3]).type(dtype)

ytrain=torch.empty(10000,1).type(dtype)

for i in range(10000):
    
    ytrain[i]=output(xtrain[i][0],xtrain[i][1],xtrain[i][2],xtrain[i][3]).type(dtype)

In [5]:
class Forward(nn.Module):
    def __init__(self):
    
        super(Forward, self).__init__()


        self.fc1 = nn.Linear(4,80)
        self.fc2 = nn.Linear(80,160)
        self.fc4=nn.Linear(160,16)
        self.fc3=nn.Linear(16,1)
        '''
        
        self.fc3 = nn.Linear(6400, 12800)

        self.fc5=nn.Linear(12800, 12800)
        self.fc6=nn.Linear(12800, 1280)
        self.fc4 = nn.Linear(1280, 784)


        self.dropout = nn.Dropout(0.3)
        '''
    def forward(self, x):
        
        x = F.relu(self.fc1(x)) # (input, negative_slope=0.2)
        
        x = F.relu(self.fc2(x))
        
        x = F.relu(self.fc4(x))
        
        x = F.relu(self.fc3(x))
        
        '''
        out =F.relu(self.fc5(x))
        out =F.relu(self.fc6(out))
        out =F.relu(self.fc4(out))
        out[out>255]=out[out>255]/100
        out[out<55]=out[out<55]*1000
        '''
        return x

In [6]:
import torch.optim as optim

learning_rate=0.001
momentum=0.05

forwardnetwork = Forward().cuda()
forwardoptimizer = optim.Adagrad(forwardnetwork.parameters(), lr=learning_rate,
                      )
print(forwardnetwork)

Forward(
  (fc1): Linear(in_features=4, out_features=80, bias=True)
  (fc2): Linear(in_features=80, out_features=160, bias=True)
  (fc4): Linear(in_features=160, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=1, bias=True)
)


In [7]:
forwardnetwork(xtrain[0:10].reshape(10,4))

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], device='cuda:0', grad_fn=<ReluBackward0>)

In [8]:
#training loop

batchsize=8

numbatches=int(len(xtrain)/batchsize)

nepochs=1000
criterion=nn.MSELoss()

for i in range(nepochs):
    
    epochloss=0
    
    for j in range(numbatches):
        
        out=forwardnetwork(xtrain[j*batchsize:(j+1)*batchsize].reshape(batchsize,4)).type(dtype)
        out=out.reshape(batchsize,1)
        #print(out)
        loss=criterion(out,ytrain[j*batchsize:(j+1)*batchsize].reshape(batchsize,1)).type(dtype)
        
        #backprop loss i.e. find dloss/dparam for each parameter and store.
        loss.backward(retain_graph=True)
        
        #clip gradients
        torch.nn.utils.clip_grad_norm_(forwardnetwork.parameters(), 100.0)
        forwardoptimizer.step()
        
        epochloss=epochloss+loss
        
        testout=forwardnetwork(xtest.reshape(1000,4))
        lossc=nn.MSELoss()
        testloss=lossc(testout,ytest)
        
    print(epochloss,testloss,i)

tensor(8.6701e+09, device='cuda:0', grad_fn=<AddBackward0>) tensor(386127.5312, device='cuda:0', grad_fn=<MseLossBackward>) 0
tensor(1.1923e+08, device='cuda:0', grad_fn=<AddBackward0>) tensor(11459.0557, device='cuda:0', grad_fn=<MseLossBackward>) 1
tensor(10262858., device='cuda:0', grad_fn=<AddBackward0>) tensor(6507.4673, device='cuda:0', grad_fn=<MseLossBackward>) 2
tensor(7242262.5000, device='cuda:0', grad_fn=<AddBackward0>) tensor(5416.7471, device='cuda:0', grad_fn=<MseLossBackward>) 3
tensor(6309479.5000, device='cuda:0', grad_fn=<AddBackward0>) tensor(4912.3633, device='cuda:0', grad_fn=<MseLossBackward>) 4
tensor(5800711., device='cuda:0', grad_fn=<AddBackward0>) tensor(4591.8550, device='cuda:0', grad_fn=<MseLossBackward>) 5
tensor(5455793., device='cuda:0', grad_fn=<AddBackward0>) tensor(4358.3506, device='cuda:0', grad_fn=<MseLossBackward>) 6
tensor(5202983., device='cuda:0', grad_fn=<AddBackward0>) tensor(4176.3701, device='cuda:0', grad_fn=<MseLossBackward>) 7
tensor(4

tensor(3754691., device='cuda:0', grad_fn=<AddBackward0>) tensor(3101.0420, device='cuda:0', grad_fn=<MseLossBackward>) 66
tensor(3751547.2500, device='cuda:0', grad_fn=<AddBackward0>) tensor(3098.4661, device='cuda:0', grad_fn=<MseLossBackward>) 67
tensor(3748509., device='cuda:0', grad_fn=<AddBackward0>) tensor(3095.8335, device='cuda:0', grad_fn=<MseLossBackward>) 68
tensor(3745472.2500, device='cuda:0', grad_fn=<AddBackward0>) tensor(3093.2122, device='cuda:0', grad_fn=<MseLossBackward>) 69
tensor(3742538., device='cuda:0', grad_fn=<AddBackward0>) tensor(3090.7732, device='cuda:0', grad_fn=<MseLossBackward>) 70
tensor(3739646.2500, device='cuda:0', grad_fn=<AddBackward0>) tensor(3088.2026, device='cuda:0', grad_fn=<MseLossBackward>) 71
tensor(3736702.7500, device='cuda:0', grad_fn=<AddBackward0>) tensor(3085.7654, device='cuda:0', grad_fn=<MseLossBackward>) 72
tensor(3733924., device='cuda:0', grad_fn=<AddBackward0>) tensor(3083.4321, device='cuda:0', grad_fn=<MseLossBackward>) 73


tensor(3621291.7500, device='cuda:0', grad_fn=<AddBackward0>) tensor(2989.2227, device='cuda:0', grad_fn=<MseLossBackward>) 131
tensor(3619883.5000, device='cuda:0', grad_fn=<AddBackward0>) tensor(2988.0415, device='cuda:0', grad_fn=<MseLossBackward>) 132
tensor(3618500.5000, device='cuda:0', grad_fn=<AddBackward0>) tensor(2986.8550, device='cuda:0', grad_fn=<MseLossBackward>) 133
tensor(3617108.7500, device='cuda:0', grad_fn=<AddBackward0>) tensor(2985.6650, device='cuda:0', grad_fn=<MseLossBackward>) 134
tensor(3615740.2500, device='cuda:0', grad_fn=<AddBackward0>) tensor(2984.4922, device='cuda:0', grad_fn=<MseLossBackward>) 135
tensor(3614314.2500, device='cuda:0', grad_fn=<AddBackward0>) tensor(2983.2756, device='cuda:0', grad_fn=<MseLossBackward>) 136
tensor(3613006.7500, device='cuda:0', grad_fn=<AddBackward0>) tensor(2982.1042, device='cuda:0', grad_fn=<MseLossBackward>) 137
tensor(3611637., device='cuda:0', grad_fn=<AddBackward0>) tensor(2980.9712, device='cuda:0', grad_fn=<Ms

tensor(3544534.5000, device='cuda:0', grad_fn=<AddBackward0>) tensor(2928.1067, device='cuda:0', grad_fn=<MseLossBackward>) 196
tensor(3543516.7500, device='cuda:0', grad_fn=<AddBackward0>) tensor(2927.3420, device='cuda:0', grad_fn=<MseLossBackward>) 197
tensor(3542526.5000, device='cuda:0', grad_fn=<AddBackward0>) tensor(2926.6228, device='cuda:0', grad_fn=<MseLossBackward>) 198
tensor(3541545., device='cuda:0', grad_fn=<AddBackward0>) tensor(2925.8301, device='cuda:0', grad_fn=<MseLossBackward>) 199
tensor(3540564.7500, device='cuda:0', grad_fn=<AddBackward0>) tensor(2925.0610, device='cuda:0', grad_fn=<MseLossBackward>) 200
tensor(3539585., device='cuda:0', grad_fn=<AddBackward0>) tensor(2924.2847, device='cuda:0', grad_fn=<MseLossBackward>) 201


KeyboardInterrupt: 

In [9]:
xtes=torch.rand(10,4).type(dtype)
ypred=forwardnetwork(xtes)
yreal=output(xtes[0][0],xtes[0][1],xtes[0][2],xtes[0][3])

print(xtes)
print(ypred)
print(yreal)
print(output(xtes[1][0],xtes[1][1],xtes[1][2],xtes[1][3]))

print(output(xtes[2][0],xtes[2][1],xtes[2][2],xtes[2][3]))

print(output(xtes[3][0],xtes[3][1],xtes[3][2],xtes[3][3]))

tensor([[0.9883, 0.8938, 0.3498, 0.5575],
        [0.7493, 0.6551, 0.6728, 0.4101],
        [0.0183, 0.3512, 0.7787, 0.2225],
        [0.6372, 0.1816, 0.0378, 0.1345],
        [0.1076, 0.0731, 0.6858, 0.6667],
        [0.3017, 0.9107, 0.1136, 0.4574],
        [0.6593, 0.8011, 0.7639, 0.9189],
        [0.9116, 0.7433, 0.8835, 0.6052],
        [0.5369, 0.6302, 0.5495, 0.3568],
        [0.6078, 0.9860, 0.6966, 0.9101]], device='cuda:0')
tensor([[16.2022],
        [14.8282],
        [11.3781],
        [13.4947],
        [13.1002],
        [13.3713],
        [16.0859],
        [16.0504],
        [13.8775],
        [15.8675]], device='cuda:0', grad_fn=<ReluBackward0>)
tensor(20.3088, device='cuda:0')
tensor(17.9708, device='cuda:0')
tensor(10.5886, device='cuda:0')
tensor(15.7538, device='cuda:0')


In [10]:
class Generator(nn.Module):
    def __init__(self):
    
        super(Generator, self).__init__()


        self.fc1 = nn.Linear(40,80)
        self.fc2 = nn.Linear(80,16)
        self.fc4=nn.Linear(16,16)
        self.fc3=nn.Linear(16,3)
        '''
        
        self.fc3 = nn.Linear(6400, 12800)

        self.fc5=nn.Linear(12800, 12800)
        self.fc6=nn.Linear(12800, 1280)
        self.fc4 = nn.Linear(1280, 784)


        self.dropout = nn.Dropout(0.3)
        '''
    def forward(self, x):
        
        x = F.relu(self.fc1(x)) # (input, negative_slope=0.2)
        
        x = F.relu(self.fc2(x))
        
        x = F.relu(self.fc4(x))
        
        x = F.relu(self.fc3(x))
        x[x>100]=x[x>100]/20
        x[x<1]=x[x<1]*10
        '''
        out =F.relu(self.fc5(x))
        out =F.relu(self.fc6(out))
        out =F.relu(self.fc4(out))
        out[out>255]=out[out>255]/100
        out[out<55]=out[out<55]*1000
        '''
        return x

In [11]:
gennetwork = Generator().cuda()
genoptimizer = optim.Adagrad(gennetwork.parameters(), lr=learning_rate,
                            )
print(gennetwork)

Generator(
  (fc1): Linear(in_features=40, out_features=80, bias=True)
  (fc2): Linear(in_features=80, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=3, bias=True)
)


In [12]:
def customloss(inp,bsize):
    
    inp=torch.cat((inp,100*torch.ones(bsize,1).type(dtype)),1).type(dtype)
    cusout=forwardnetwork(inp.reshape([bsize,4])).type(dtype)
    
    cusout=cusout.reshape([bsize,1]).type(dtype)
    cusideal=788*torch.ones([bsize,1]).type(dtype)
    
    cuscriterion= nn.MSELoss()
    
    loss= cuscriterion(cusout,cusideal).type(dtype)
    
    return loss

In [13]:
xgarb=torch.rand([100,40]).type(dtype)

In [18]:
batchsize=2

batches=len(xgarb)/batchsize

epochs=1000

for i in range(epochs):
    
    epochloss=0
        
    for j in range(int(batches)):
        
        
        #print(Ytensor)
        out=gennetwork(xgarb[j*batchsize:(j+1)*batchsize].reshape([batchsize,40])).type(dtype)
        
        if out[0][0]>100:
            print('done')
            break
        #compute loss
        loss = customloss(out,batchsize).type(dtype)


        #backprop loss i.e. find dloss/dparam for each parameter and store.
        loss.backward(retain_graph=True)
        
        #clip gradients
        torch.nn.utils.clip_grad_norm_(gennetwork.parameters(), 1.0)
        genoptimizer.step()
        
        epochloss=epochloss+loss
    
    if out[0][0]>100:
            print('done')
            break
    print(epochloss,i)

tensor(3986180.2500, device='cuda:0', grad_fn=<AddBackward0>) 0
tensor(3975526.5000, device='cuda:0', grad_fn=<AddBackward0>) 1
tensor(3964880.5000, device='cuda:0', grad_fn=<AddBackward0>) 2
tensor(3954243.7500, device='cuda:0', grad_fn=<AddBackward0>) 3
tensor(3943612.2500, device='cuda:0', grad_fn=<AddBackward0>) 4
tensor(3932987.2500, device='cuda:0', grad_fn=<AddBackward0>) 5
tensor(3922360.5000, device='cuda:0', grad_fn=<AddBackward0>) 6
tensor(3911729., device='cuda:0', grad_fn=<AddBackward0>) 7
tensor(3901100.2500, device='cuda:0', grad_fn=<AddBackward0>) 8
tensor(3890479.2500, device='cuda:0', grad_fn=<AddBackward0>) 9
tensor(3879861.7500, device='cuda:0', grad_fn=<AddBackward0>) 10
tensor(3869242.7500, device='cuda:0', grad_fn=<AddBackward0>) 11
tensor(3858623., device='cuda:0', grad_fn=<AddBackward0>) 12
tensor(3848009.7500, device='cuda:0', grad_fn=<AddBackward0>) 13
tensor(3837398., device='cuda:0', grad_fn=<AddBackward0>) 14
tensor(3826792.2500, device='cuda:0', grad_fn=<

tensor(2667307., device='cuda:0', grad_fn=<AddBackward0>) 129
tensor(2657687.7500, device='cuda:0', grad_fn=<AddBackward0>) 130
tensor(2648079.5000, device='cuda:0', grad_fn=<AddBackward0>) 131
tensor(2638480.2500, device='cuda:0', grad_fn=<AddBackward0>) 132
tensor(2628892.2500, device='cuda:0', grad_fn=<AddBackward0>) 133
tensor(2619315.7500, device='cuda:0', grad_fn=<AddBackward0>) 134
tensor(2609750.5000, device='cuda:0', grad_fn=<AddBackward0>) 135
tensor(2600198.7500, device='cuda:0', grad_fn=<AddBackward0>) 136
tensor(2590657.5000, device='cuda:0', grad_fn=<AddBackward0>) 137
tensor(2581128.2500, device='cuda:0', grad_fn=<AddBackward0>) 138
tensor(2571613.2500, device='cuda:0', grad_fn=<AddBackward0>) 139
tensor(2562110., device='cuda:0', grad_fn=<AddBackward0>) 140
tensor(2552617.7500, device='cuda:0', grad_fn=<AddBackward0>) 141
tensor(2543136.5000, device='cuda:0', grad_fn=<AddBackward0>) 142
tensor(2533668.5000, device='cuda:0', grad_fn=<AddBackward0>) 143
tensor(2524211., d

In [19]:
loss

tensor(47441.8828, device='cuda:0', grad_fn=<MseLossBackward>)

In [20]:
print(out)
print(output(out[0][0],out[0][1],out[0][2],100*torch.ones(1).type(dtype)))

tensor([[100.0423,   0.0000,  53.7249],
        [ 81.6447,   0.0000,  43.8124]], device='cuda:0',
       grad_fn=<IndexPutBackward>)
tensor([777.5806], device='cuda:0', grad_fn=<AddBackward0>)


In [21]:
output(100,100,100,100*torch.ones(1).type(dtype))

tensor([787.3434], device='cuda:0')

In [ ]:

xtens0=torch.linspace(0,10,1000).type(dtype)
xtens1=torch.linspace(2,54,1000).type(dtype)
xtens2=torch.linspace(1,100,1000).type(dtype)
xtens3=torch.linspace(10,100,1000).type(dtype)

ytens=torch.empty(1000,1).type(dtype)

for i in range(1000):
    
    ytens[i]=out(xtens0[i],xtens1[i],xtens2[i],xtens3[i]).type(dtype)

print(ytens)